# User study experiments

In [1]:
%load_ext autoreload
%autoreload 2
import numpy as np
from src.motion_refiner_4D import Motion_refiner, MAX_NUM_OBJS
from src.config import *
# base_folder = "/home/tum/data/"
# base_folder = "/home/arthur/local_data/" 

traj_n = 40
mr = Motion_refiner(load_models=True ,traj_n = traj_n, locality_factor=True, clip_only=False,load_precomp_emb=True)
feature_indices, obj_sim_indices, obj_poses_indices, traj_indices = mr.get_indices()
embedding_indices = np.concatenate([feature_indices,obj_sim_indices, obj_poses_indices])

/home/arthur/.conda/envs/py38_cu11_2/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


loading BERT model... 

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertModel: ['vocab_projector.weight', 'vocab_projector.bias', 'vocab_transform.weight', 'vocab_layer_norm.weight', 'vocab_transform.bias', 'vocab_layer_norm.bias']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


done
loading CLIP model... done
loading precomputed CLIP text embbedings... done
loading precomputed CLIP img embbedings... done
DEVICE:  cuda


In [2]:
dataset_name = "latte_100k_lf"

X,Y, data = mr.load_dataset(dataset_name, filter_data = True, base_path=base_folder+"data/")
X_train, X_test, X_valid, y_train, y_test, y_valid, indices_train, indices_test, indices_val = mr.split_dataset(X, Y, test_size=0.2, val_size=0.1)

loading dataset:  latte_100k_lf ...done
raw X: (100000, 953) 	Y: (100000, 160)
filtered X: (96718, 953) 	Y: (96718, 160)


2022-09-15 22:51:53.843202: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-09-15 22:51:53.844205: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-09-15 22:51:53.844675: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-09-15 22:51:53.845321: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags

Train X: (67702, 953) 	Y: (67702, 160)
Test  X: (19344, 953) 	Y: (19344, 160)
Val   X: (9672, 953) 	Y: (9672, 160)


### load datasets with predicted data

In [3]:
data_pred = mr.load_data("testpred_100k_latte_f", base_path=data_folder)
data_no_language = mr.load_data("test_no_language_100k_latte_f", base_path=data_folder)
data_2d = mr.load_data("pred2D_100k_latte_f", base_path=data_folder)


### 2D model

In [4]:
from src.simple_TF_continuos import *

model_path = "./models/"
model_name = "refined_refined_TF&num_layers_enc:2&num_layers_dec:4&d_model:256&dff:512&num_heads:8&dropout_rate:0.1&wp_d:2&bs:64&dense_n:512&num_dense:3&concat_emb:True&features_n:777&optimizer:RMSprop&norm_layer:True&activation:tanh.h5"

model_file = model_path+model_name
model_2d = load_model(model_file)
compile(model_2d)

{'num_layers_enc': 2, 'num_layers_dec': 4, 'd_model': 256, 'dff': 512, 'num_heads': 8, 'dropout_rate': 0.1, 'wp_d': 2, 'bs': 64, 'dense_n': 512, 'num_dense': 3, 'concat_emb': True, 'features_n': 777, 'optimizer': 'RMSprop', 'norm_layer': True, 'activation': 'tanh'}
loading weights:  ./models/refined_refined_TF&num_layers_enc:2&num_layers_dec:4&d_model:256&dff:512&num_heads:8&dropout_rate:0.1&wp_d:2&bs:64&dense_n:512&num_dense:3&concat_emb:True&features_n:777&optimizer:RMSprop&norm_layer:True&activation:tanh.h5


In [24]:
from src.functions import *
from src.traj_utils import *
from src.motion_refiner import Motion_refiner as Motion_refiner2D

mr2d = Motion_refiner2D(load_models=True)


Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertModel: ['vocab_transform.bias', 'vocab_projector.bias', 'vocab_projector.weight', 'vocab_layer_norm.weight', 'vocab_layer_norm.bias', 'vocab_transform.weight']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [43]:
def apply_interaction(mr, traj, obj_poses_, text, obj_names, n_obj=3):
    mod = 1
    p1 = np.array([0.1, 0.1])-0.5
    p2 = np.array([0.9, 0.9])-0.5
    traj_np = traj
    if isinstance(traj, list):
        traj_np = np.array(traj)
    obj_poses = obj_poses_
    if isinstance(obj_poses_, list):
        obj_poses = np.array(obj_poses_)
    traj_raw = traj_np[::mod, :2]
    t0 = traj_raw[0, :]
    tf = traj_raw[-1, :]
    traj_raw_n, obj_poses_new = interpolate_2points(
        traj_raw, p1, p2, objs=obj_poses.T)

    d = np2data(traj_raw_n, obj_names,
                obj_poses_new.T, text, output_traj=None)
    # X, _ = mr.prepare_data(d, label=False)
    pred, traj_in = mr.apply_interaction(
        model_2d, d[0], text,  label=False)
    # print(pred)
    # print(pred.shape)
    # print(t0, tf)

    new_traj_simple = interpolate_2points(pred[0, :, :], t0, tf)

    new_traj_wp = fit_wps_to_traj(new_traj_simple, traj_raw)

    constraints = np.ones([traj_raw.shape[0]])*0.15

    new_traj_wp_cnt = mr.follow_hard_constraints(
        traj_raw, new_traj_wp, constraints)

    new_traj_wp_scaled = mr.addapt_to_hard_constraints(
        traj_raw, new_traj_wp, constraints)

    return new_traj_wp, pred[0, :, :], obj_poses_new.T

data_2d = []
for d_ in data[:10000]:
    if len(d_["obj_names"]) == 3:
        d=d_.copy()
    
        traj, obj_poses, text, obj_names = d["input_traj"], d["obj_poses"],d["text"],d["obj_names"]
        traj = np.array(traj)
        obj_poses = np.array(obj_poses)
        try:
            new_traj_wp, pred, obj_poses_new = apply_interaction(mr2d, traj[:,:2], obj_poses[:3,:2].T, text, obj_names, n_obj=3)
            traj_new = traj.copy()
            traj_new[:,:2] = new_traj_wp[:,:2]
            d["output_traj"] = traj_new
            data_2d.append(d)
        except:
            pass
    # print(d["output_traj"])
print(len(data_2d))
mr.save_data(data_2d,data_name="pred2D_100k_latte_f", base_path=data_folder)

1900


## opposit of Groud Truth

In [4]:
data_opposit = []
for d_ in data:
    d = d_.copy()
    traj_in = np.array(d["input_traj"])
    traj_out = np.array(d["output_traj"])
    delta = traj_out-traj_in
    new_traj = traj_in - delta
    d["output_traj"] = new_traj.tolist()
    data_opposit.append(d)


### Load model for real time interactions

In [5]:
from src.TF4D_mult_features import *
model_path = models_folder+"FINAL_dataset_size_aug_fixsteps/"

model_name = "TF-num_layers_enc:1-num_layers_dec:5-d_model:400-dff:512-num_heads:8-dropout_rate:0.1-wp_d:4-num_emb_vec:4-bs:16-dense_n:512-num_dense:3-concat_emb:False-features_n:793-optimizer:adam-norm_layer:True-activation:tanh-loss:mse-sf:0.5-augment:1.h5"
model_file = model_path+model_name

model = load_model(model_file, delimiter="-")

{'num_layers_enc': 1, 'num_layers_dec': 5, 'd_model': 400, 'dff': 512, 'num_heads': 8, 'dropout_rate': 0.1, 'wp_d': 4, 'num_emb_vec': 4, 'bs': 16, 'dense_n': 512, 'num_dense': 3, 'concat_emb': False, 'features_n': 793, 'optimizer': 'adam', 'norm_layer': True, 'activation': 'tanh', 'loss': 'mse', 'sf': 0.5, 'augment': 1}
loading weights:  /home/arthur/local_data/models/FINAL_dataset_size_aug_fixsteps/TF-num_layers_enc:1-num_layers_dec:5-d_model:400-dff:512-num_heads:8-dropout_rate:0.1-wp_d:4-num_emb_vec:4-bs:16-dense_n:512-num_dense:3-concat_emb:False-features_n:793-optimizer:adam-norm_layer:True-activation:tanh-loss:mse-sf:0.5-augment:1.h5


## Experiment

In [67]:
%matplotlib qt

%load_ext autoreload
%autoreload 2

from matplotlib import pyplot as plt
import numpy as np
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import animation
from matplotlib.widgets import Button, TextBox, RadioButtons

import re
from src.functions import *
from matplotlib import rc
import textwrap
import random
import json
from src.config import *
import datetime

w = textwrap.TextWrapper(width=60,break_long_words=False,replace_whitespace=False)

# rc('text', usetex=True)

class User_study_interface():

    def __init__(self, data_ditributions, samples_per_data=1, interaction_samples=1, dis_names=None, model=None):
        """data_ditributions: list, with the different data distributions
        samples_per_data: int number of samples per distribution to present the user
        dis_names: list of with the distributuions names"""

        
        self.model = model
        self.dis_names = [str(i) for i in range(len(data_ditributions))] if dis_names is None else dis_names
        self.num_samples = samples_per_data*len(data_ditributions) # per ditribution
        exp_data = []
        exp_data_indices = []
        exp_sample_indices = []

        self.interaction_text = {}
        self.samples_inices = {}
        self.user_answers = {}
        self.interaction_samples = interaction_samples
        for i,data_dis in enumerate(data_ditributions):

            samples_inices = random.choices(range(len(data_dis)), k=samples_per_data)
            # print(self.dis_names[i],samples_inices)
            exp_data = exp_data + np.array(data_dis)[samples_inices].tolist()
            
            # exp_data = exp_data + random.choices(data_dis, k=samples_per_data)
            self.samples_inices[self.dis_names[i]] = samples_inices
            
            exp_data_indices = exp_data_indices + [i]*samples_per_data
            exp_sample_indices = exp_sample_indices + samples_inices


        index_shuf = list(range(len(exp_data)))
        random.shuffle(index_shuf)
        # print(index_shuf)
        self.exp_data = [exp_data[i].copy() for i in index_shuf]
        self.exp_data_indices = [exp_data_indices[i] for i in index_shuf]
        self.exp_sample_indices = [exp_sample_indices[i] for i in index_shuf]


        data_dis = data_ditributions[0]
        samples_inices = random.choices(range(len(data_dis)), k=interaction_samples)
        self.exp_data  = self.exp_data + np.array(data_dis)[samples_inices].tolist()
        self.samples_inices["interaction"] = samples_inices
        self.exp_data_indices = self.exp_data_indices + [i]*interaction_samples
        self.exp_sample_indices = self.exp_sample_indices + samples_inices

        self.init_setup()

    def init_setup(self):
        self.fig = plt.figure(figsize=(8,13))
        self.fig.add_subplot(1,1,1,projection='3d')
        self.ax = plt.gca(projection='3d')
        # self.ax.set_xlabel('X axis')
        # self.ax.set_ylabel('Y axis')
        # self.ax.set_zlabel('Z axis')
        self.ax.axes.xaxis.set_ticklabels([])
        self.ax.axes.yaxis.set_ticklabels([])
        self.ax.axes.zaxis.set_ticklabels([])

        self.ani=None
        self.lines = []
        self.tip_marker = []
        self.objs_scatter = []
        self.objs_text = []
        self.colors = []
        self.enable_save = False
        self.user_data = {"name":"", "age":""}
        self.eval_text_list = ['yes, much better', 'yes, a bit better' , 'same', 'No, a bit wrong', 'No, totally wrong']

        self.sample_i =1

        self.alert_text = None
        self.count_text = self.ax.text2D(0.05, 0.95, str(self.sample_i)+"/"+str(self.num_samples+self.interaction_samples), transform=self.ax.transAxes)

        self.plot_sample(self.exp_data[self.sample_i-1])
        self.setup_bts()
        self.reset_bts()
        self.plot_markers()

        plt.show()

    def update(self, num, trajs, lines, tips):

        for i, line in enumerate(lines):
            line.set_data(trajs[i][:2,:num])
            line.set_3d_properties(trajs[i][2,:num])
        for i, tip in enumerate(tips):
            n = max(num-1, 0)
            tip.set_data(trajs[i][:2,n:n+1])
            tip.set_3d_properties(trajs[i][2,n:n+1])
            
    def next_cb(self, event):
        if self.sample_i in self.user_answers.keys():
            self.sample_i += 1
            if self.sample_i > self.num_samples:
                if self.sample_i <= self.num_samples+self.interaction_samples:
                    self.reset()
                    self.plot_intercative()
                else:
                    self.reset()
                    self.plot_final_screen()
            
            else:
                self.plot_sample(self.exp_data[self.sample_i-1])
                self.reset_bts()

        else:
            if not self.alert_text is None:
                self.alert_text.remove()
            self.alert_text = self.ax.text2D(0.50, -0.1,"Please answer the question first!", transform=self.ax.transAxes, color="red")
        

    def prev_cb(self, event):
        if self.sample_i > 1:
            self.sample_i -= 1
            if self.sample_i > self.num_samples:
                if self.sample_i <= self.num_samples+self.interaction_samples:
                    self.reset()
                    self.plot_intercative()
                else:
                    pass
            else:
                self.plot_sample(self.exp_data[self.sample_i-1])

                self.reset_bts()


    def eval_cb(self, labels):
        self.user_answers[self.sample_i] = self.eval_text_list.index(labels)

        # self.radio.remove()
    

    def plot_final_screen(self):
        self.final_bg = plt.axes([0.05, 0.05, 0.9, 0.9], facecolor="white")


        self.final_bg.set_xticks([])
        self.final_bg.set_yticks([])
        final_text = self.final_bg.text(0.05, 0.7,"Thank you for helping us :)", transform=self.final_bg.transAxes, color="Black", fontweight="bold", fontsize=27)

        self.save_text = self.final_bg.text(0.2, 0.63," ", transform=self.final_bg.transAxes, color="white")

        self.ax_name_box = plt.axes([0.2, 0.5, 0.4, 0.04])
        self.name_box = TextBox(self.ax_name_box, 'your name:', initial=self.user_data["name"])
        self.name_box.on_submit(lambda val: self.log_entry(val,"name"))

        self.ax_age_box = plt.axes([0.2, 0.4, 0.4, 0.04])
        self.age_box = TextBox(self.ax_age_box, 'your age:', initial=self.user_data["age"])
        self.age_box.on_submit(lambda val: self.log_entry(val,"age"))

        self.axsave = plt.axes([0.2, 0.2, 0.20, 0.075])
        self.btsave = Button(self.axsave, 'SAVE and EXIT')
        self.btsave.on_clicked(self.save_data)
    
    def plot_intercative(self):
        # self.interactive_bg = plt.axes([0.05, 0.7, 0.9, 0.3], facecolor="white")
        # self.interactive_bg.set_xticks([])
        # self.interactive_bg.set_yticks([])
        
        # final_text = self.final_bg.text(0.05, 0.7,"Thank you for helping us :)", transform=self.final_bg.transAxes, color="Black", fontweight="bold", fontsize=27)

        # self.save_text = self.final_bg.text(0.2, 0.63," ", transform=self.final_bg.transAxes, color="white")

        self.nl_box_ax = plt.axes([0.2, 0.85, 0.4, 0.04])
        init_text = "" if not self.sample_i in self.interaction_text.keys() else self.interaction_text[self.sample_i]
        self.nl_box = TextBox(self.nl_box_ax, '', initial=init_text )
        self.nl_box.on_submit(self.register_interaction)

        self.axmodify = plt.axes([0.6, 0.85, 0.20, 0.040])
        self.btmodify = Button(self.axmodify, 'modify trajectory')
        self.btmodify.on_clicked(self.predict)


        plot_out = self.sample_i in self.interaction_text.keys()
        self.plot_sample(self.exp_data[self.sample_i-1],plot_out=plot_out)
        self.ax.set_title(" Type your instruction:\n\n\n\n\n\n", fontsize=18,  fontname="Times New Roman")
        self.reset_bts()

    def register_interaction(self, text):
        self.interaction_text[self.sample_i] = text

    def predict(self, event):
        if self.sample_i in self.interaction_text.keys():
            d = self.exp_data[self.sample_i-1]
            pred, traj = mr.apply_interaction(self.model, d, self.interaction_text[self.sample_i],  label=False, images=None)
            self.pred=pred[0]
            self.exp_data[self.sample_i-1]["output_traj"] = pred[0].tolist()
            # x_t = (mr.prepare_x(X_test), list_to_wp_seq(y_test,d=4), X_test[:,embedding_indices])
            # self.pred = generate(model ,x_t, traj_n=traj_n).numpy()
            print(self.pred.shape)
            self.plot_sample(self.exp_data[self.sample_i-1],new_pred=True)
            self.ax.set_title(" Type your instruction:\n\n\n\n\n\n", fontsize=18,  fontname="Times New Roman")
            self.reset_bts()
        else:
            print("write first")

    def log_entry(self, val, k):
        self.user_data[k] = str(val)

        self.enable_save = True
        for key in self.user_data.keys():
            if self.user_data[key] == "":
                self.enable_save = False

    def save_data(self, name):
        if self.enable_save:
            if not os.path.exists(user_study_folder):
                print("User study folder not found:", user_study_folder)

            summary = {d_name:{a:0 for a in self.eval_text_list} for d_name in self.dis_names}
            answers_per_distribution = {d_name:{a:[] for a in self.eval_text_list} for d_name in self.dis_names}

            for i,(k,v) in enumerate(self.user_answers.items()):
                print("V",v, "  K: ",k)
                dis = self.dis_names[self.exp_data_indices[int(k-1)]]
                answer = self.eval_text_list[v]
                summary[dis][answer]= summary[dis][answer] + 1
                print(self.exp_sample_indices[i],"\t")
                
                answers_per_distribution[dis][answer] = answers_per_distribution[dis][answer] + [self.exp_sample_indices[i]]

            data_to_save = {"summary":summary,
                            "answers_per_distribution":answers_per_distribution,
                            "num_samples":self.num_samples,
                            "dis_names":self.dis_names,
                            "exp_data":self.exp_data,
                            "exp_data_indices":self.exp_data_indices,
                            "exp_sample_indices":self.exp_sample_indices,
                            "user_answers":self.user_answers,
                            "interaction_text":self.interaction_text
                            }
            # data_to_save
            print(self.user_answers)
            with open( os.path.join(user_study_folder, self.user_data["name"]+"_"+self.user_data["age"] +"_"+ datetime.datetime.now().strftime("%Y%m%d-%H%M%S") +'.json'), 'w') as f:
                json.dump(data_to_save, f)
            plt.close()
        else: 
            if not self.save_text is None:
                self.save_text.remove()
            self.save_text = self.final_bg.text(0.2, 0.63,"Please complete all the fields first", transform=self.final_bg.transAxes, color="red")
    
    def replay_from_file(self,user_name):
        print(user_name)
        u = {}
        with open(os.path.join(user_study_folder,user_name), 'r', encoding='utf-8') as f:
            u = json.load(f)

        self.num_samples = u["num_samples"]
        self.dis_names = u["dis_names"]
        self.exp_data = u["exp_data"]
        self.exp_data_indices = u["exp_data_indices"]
        self.exp_sample_indices = u["exp_sample_indices"]
        # self.interaction_text = "interaction_text"
        self.sample_i =1

        for k,v in u["interaction_text"].items():
            self.interaction_text[int(k)]=v
        for k,v in u["user_answers"].items():
            self.user_answers[int(k)]=int(v)
        self.init_setup()
        # self.user_answers = u["user_answers"]

        # for k,v in u.items():
        #     print(k,v)
        print(self.user_answers)
        print(self.sample_i)

        if self.sample_i in self.user_answers.keys() or str(self.sample_i) in self.user_answers.keys():
            print("A")
            print(self.user_answers[self.sample_i])
            print(type(self.user_answers[self.sample_i]))




    def setup_bts(self):
        
        self.alert_text = self.ax.text2D(0.70, 0.05, "alert", transform=self.ax.transAxes, color="white")
        axcolor = 'lightgoldenrodyellow'

        self.rax = plt.axes([0.15, 0.05, 0.2, 0.15],
                    facecolor=axcolor)
                    
        self.rax.set_title("Does the modified trajectory \n follows the intruction?")

        self.axnext = plt.axes([0.66, 0.05, 0.15, 0.075])
        if self.sample_i < self.num_samples+self.interaction_samples:
            self.bnext = Button(self.axnext, 'Next')
        elif self.sample_i == self.num_samples+self.interaction_samples:
            self.bnext = Button(self.axnext, 'FINISH!')
        

        self.bnext.on_clicked(self.next_cb)

        self.axprev = plt.axes([0.5, 0.05, 0.15, 0.075])
        if self.sample_i > 1:
            self.bprev = Button(self.axprev, 'Previous')

        else:
            self.bprev = Button(self.axprev, 'Previous',color="white")

        self.bprev.on_clicked(self.prev_cb)

        # adjust radio buttons
        self.radio = RadioButtons(self.rax, self.eval_text_list,
                            active=[True,False,False,False],
                            activecolor='r')

        self.radio.on_clicked(self.eval_cb)
        if self.sample_i in self.user_answers.keys() or str(self.sample_i) in self.user_answers.keys():
            print("A")
            self.radio.set_active(self.user_answers[self.sample_i])

    def reset_bts(self):
        self.axprev.remove()
        self.axnext.remove()
        self.rax.remove()
        self.setup_bts()
        
    def plot_sample(self, d, new_pred=False, plot_out=True, no_title=False):
        
        self.reset()
        
        pts = np.asarray(d["input_traj"])
        if new_pred:
            pts_new = self.pred
        else:
            pts_new = np.asarray(d["output_traj"])
        
        text = d["text"]
        obj_names = np.asarray(d["obj_names"])
        obj_pt = np.asarray(d["obj_poses"])
        change_type = d["change_type"]
        image_paths = d["image_paths"]

        objs  = {}
        for x,y,z,name in zip(obj_pt[:,0],obj_pt[:,1],obj_pt[:,2],obj_names):
            objs[name] = {"value":{"obj_p":[x,y,z]}}

        new_pts_list = [pts_new]
        if d["output_traj"] is None:
            new_pts_list = []

        N = 100
        dt = 0.02
        traj_original = incorporate_speed(pts,dt=dt, N=N)
        traj_new = incorporate_speed(pts_new,dt=dt, N=N)
        # fig = plot_samples(text,pts,new_pts_list, objs=objs,fig=figure, show= False, plot_speed=False, labels=["modified"])
        # if not plot_out:
        #     traj_new = traj_original.copy()
        self.plot_objs(objs)
        if plot_out:
            self.trajs = [traj_original.T, traj_new.T]
        else:
            self.trajs = [traj_original.T] 

        self.plot_trajs(traj_original, traj_new,plot_out=plot_out)

        if not no_title:
            self.ax.set_title("INSTRUCTION:\n\n"+  '\n'.join(w.wrap(text))  , fontsize=18,  fontname="Times New Roman")
        else:

            self.ax.set_title(" ", fontsize=18,  fontname="Times New Roman")
        set_axes_equal(self.ax)

        self.animate(N=100,plot_out=plot_out)

    def plot_objs(self, objs):
        for i,(name,v) in enumerate(objs.items()):
            x,y,z = v["value"]["obj_p"]
            
            color = self.colors[i] if i < len(self.colors)-1 else "#"+''.join([random.choice('0123456789AB') for j in range(6)])    

            sc = self.ax.scatter(x,y,z, color=color, s=50)
            # print(dir(sc))
            self.objs_scatter.append(sc)
            t = self.ax.text(x, y, z, name, 'x', color=color, ha='center', fontweight="bold")
            self.objs_text.append(t)

    def plot_trajs(self, pts, pts_new, plot_out=True):

        alpha = 0.1
        color_original = "red"
        color_modified = "blue"

        x_init, y_init, z_init = pts[:,0],pts[:,1],pts[:,2]
        x_new, y_new, z_new= pts_new[:,0],pts_new[:,1],pts_new[:,2]
        

        line3, = self.ax.plot(x_init, y_init, z_init,alpha=0.9,color=color_original, label="Original")
        self.lines.append(line3)
        if plot_out:
            line4, = self.ax.plot(x_new, y_new, z_new,alpha=0.9, color=color_modified, label="Modified")
            self.lines.append(line4)
        handles, labels = self.ax.get_legend_handles_labels()
        self.ax.legend(handles[::-1], labels[::-1])

        line1, = self.ax.plot(x_init, y_init, z_init,alpha=alpha,color=color_original, label="Original")
        self.lines.append(line1)
        if plot_out:
            line2, = self.ax.plot(x_new, y_new, z_new,alpha=alpha, color=color_modified, label="Modified")
            self.lines.append(line2)

        tip_marker1, = self.ax.plot(x_init[0:1], y_init[0:1], z_init[0:1], lw=2, c=color_original, marker='o')
        self.tip_marker.append(tip_marker1)
        if plot_out:
            tip_marker2, = self.ax.plot(x_new[0:1], y_new[0:1], z_new[0:1], lw=2, c=color_modified, marker='o')
            self.tip_marker.append(tip_marker2)

    def plot_markers(self):

        fs=30
        dist = 0.5
        alpha = 0.2
        color = 'grey'
        self.ax.text(0,dist, 0, "front", 'x',color=color, alpha=alpha, fontsize=fs, ha='center', va='center')
        self.ax.text(-dist,0, 0, "left", 'y',color=color, alpha=alpha, fontsize=fs, ha='center', va='center')
        self.ax.text(0,0, -dist, "bottom", 'x',color=color, alpha=alpha, fontsize=fs, ha='center', va='center')

        # self.ax.text(0,-dist, 0, "back", 'x',color='red', alpha=alpha, fontsize=fs, ha='center', va='center')
        # self.ax.text(dist,0, 0, "right", 'y',color='red', alpha=alpha, fontsize=fs, ha='center', va='center')
        # self.ax.text(0,0, dist, "up", 'x',color='red', alpha=alpha, fontsize=fs, ha='center', va='center')


    def reset(self):
        self.count_text.remove()
        self.count_text = self.ax.text2D(0.05, 0.95, str(self.sample_i)+"/"+str(self.num_samples+self.interaction_samples), transform=self.ax.transAxes)
        
        try:
            if not self.alert_text is None:
                self.alert_text.remove()
        except:
            pass

        try:
            self.interactive_bg.remove()
        except:
            pass

        for i in range(len(self.lines)):
            self.lines[i].remove()

        for i in range(len(self.tip_marker)):
            self.tip_marker[i].remove()

        for i in range(len(self.objs_text)):
            # print(self.objs_scatter[i])
            # print("\n",self.objs_scatter[i])
            self.objs_scatter[i].remove()
            self.objs_text[i].remove()

        self.objs_scatter = []
        self.objs_text = []
        self.lines = []
        self.tip_marker = []



    def animate(self, N=100,plot_out=True):
        n_lines = 2 if plot_out else 1
        self.ani = animation.FuncAnimation(self.fig, self.update, N, fargs=(self.trajs,self.lines[:n_lines], self.tip_marker[:n_lines]), interval=1000/N,cache_frame_data=False, blit=False)

    
# user_study = User_study_interface([data], dis_names=["data_pred"],samples_per_data=1, interaction_samples=3, model=model)


user_study = User_study_interface([data,data_pred, data_no_language,data_2d, data_opposit], dis_names=["Ground Truth", "Ours", "No_language","2D_only","GT opposit"],samples_per_data=5, interaction_samples=5,model=model)
# user_study.replay_from_file("")
plt.show()




The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [7]:
user_study.replay_from_file("Sabrinaaaaaaaaaaaaaaaaaa_25_20220915-224920.json")


Sabrinaaaaaaaaaaaaaaaaaa_25_20220915-224920.json
A
A
{1: 1, 2: 0, 3: 1, 4: 3, 5: 4, 6: 0, 7: 4, 8: 4, 9: 4, 10: 4, 11: 1, 12: 4, 13: 4, 14: 0, 15: 4, 16: 4, 17: 0, 18: 0, 19: 0, 20: 4, 21: 3, 22: 3, 23: 4, 24: 4, 25: 4, 26: 2, 27: 2, 28: 2, 29: 2, 30: 2}
1
A
1
<class 'int'>


A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
DONE - computing textual embeddings (1, 768)


1it [00:00, 1683.78it/s]


DONE - computing similarity vectors 
DONE - concatenating 
(1, 953)


100%|██████████| 39/39 [00:06<00:00,  6.45it/s]


...

(40, 4)
A
A
DONE - computing textual embeddings (1, 768)


1it [00:00, 3927.25it/s]


DONE - computing similarity vectors 
DONE - concatenating 
(1, 953)


100%|██████████| 39/39 [00:01<00:00, 19.70it/s]


(40, 4)
A
V 1   K:  1
620 	
V 0   K:  2
36700 	
V 1   K:  3
16304 	
V 3   K:  4
16952 	
V 4   K:  5
25411 	
V 0   K:  6
503 	
V 4   K:  7
23853 	
V 4   K:  8
56539 	
V 4   K:  9
1737 	
V 4   K:  10
6102 	
V 1   K:  11
15010 	
V 4   K:  12
6087 	
V 4   K:  13
86835 	
V 0   K:  14
871 	
V 4   K:  15
5179 	
V 4   K:  16
54294 	
V 0   K:  17
53867 	
V 0   K:  18
95704 	
V 0   K:  19
61899 	
V 4   K:  20
1245 	
V 3   K:  21
751 	
V 3   K:  22
66214 	
V 4   K:  23
4430 	
V 4   K:  24
4081 	
V 4   K:  25
18239 	
V 4   K:  26
65795 	
V 4   K:  27
11079 	
V 1   K:  28
85574 	
V 0   K:  29
72623 	
V 4   K:  30
74337 	
{1: 1, 2: 0, 3: 1, 4: 3, 5: 4, 6: 0, 7: 4, 8: 4, 9: 4, 10: 4, 11: 1, 12: 4, 13: 4, 14: 0, 15: 4, 16: 4, 17: 0, 18: 0, 19: 0, 20: 4, 21: 3, 22: 3, 23: 4, 24: 4, 25: 4, 26: 4, 27: 4, 28: 1, 29: 0, 30: 4}


### visualize results

In [50]:

dis_names = ["Ground Truth", "Ours", "No_language","2D_only","GT opposit","Ours w/ user text"]
answers=['yes, much better', 'yes, a bit better', 'same', 'No, a bit wrong', 'No, totally wrong']

text_interactions=[]

total={d_n:{} for d_n in dis_names}
plot_data={d_n:0 for d_n in dis_names}
num_users = 0
for user in os.listdir(user_study_folder):
    if 'Sabrina' in user:
        continue
    user_file = os.path.join(user_study_folder,user)
    u = {}
    with open(user_file, 'r', encoding='utf-8') as f:
        u = json.load(f)

    for i,dis_i in enumerate(u["exp_data_indices"]):
        if i < len(u["exp_data_indices"])-5:
            d_n = dis_names[dis_i]
        else:
            d_n = dis_names[-1]
        a = answers[int(u["user_answers"][str(i+1)])]
        if not a in total[d_n].keys():
            total[d_n][a] = 0
        total[d_n][a]=total[d_n][a]+ 1
    
    # print(u.keys())
    # print(u['interaction_text'])
    text_interactions =text_interactions+list(u['interaction_text'].values())
    num_users += 1
    # for d_n in dis_names:
    #     for a in answers:
    #         if not a in total[d_n].keys():
    #             total[d_n][a] = 0
    #         total[d_n][a]=total[d_n][a]+ int(u["summary"][d_n][a])
    # print(u["user_answers"],"\n")
    # print(u["exp_data_indices"],"\n")
print(total)
print("\n",text_interactions)
plot_dis_names = ["Ground Truth", "Ours","GT opposit", "No_language","2D_only",]
plot_dis_names_legend = ["Ground Truth", "Ours","Ground Fake", "No language","Projected 2D model"]

for d_n in plot_dis_names:
    print(d_n,total[d_n], "\n")

barWidth = 0.15
fig = plt.subplots(figsize =(12, 8))

plt.grid(axis = 'y', linewidth = 0.5)
br = [np.arange(len(plot_dis_names))]

colors=["#4f963b","#839642","#f5b342","#8a4f0b","#8c152d"]
for i,a in enumerate(answers):
    v = [total[d_n][a]*100/(5*num_users) for d_n in plot_dis_names]
    plt.bar(br[i], v, color =colors[i], width = barWidth,
        edgecolor ='grey', label =a)
    br.append([x + barWidth for x in br[i]])
    
plt.xlabel('', fontsize = 15)
plt.ylabel('percentage of answers  [%]', fontsize = 15)#fontweight ='bold', 
plt.xticks([r + barWidth for r in range(len(plot_dis_names))],
        plot_dis_names_legend)
plt.legend()
plt.show()




{'Ground Truth': {'yes, much better': 14, 'yes, a bit better': 21, 'same': 10, 'No, a bit wrong': 4, 'No, totally wrong': 1}, 'Ours': {'yes, much better': 11, 'yes, a bit better': 22, 'No, totally wrong': 1, 'same': 13, 'No, a bit wrong': 3}, 'No_language': {'yes, much better': 12, 'No, a bit wrong': 13, 'yes, a bit better': 5, 'No, totally wrong': 17, 'same': 3}, '2D_only': {'yes, a bit better': 13, 'same': 6, 'No, totally wrong': 9, 'yes, much better': 9, 'No, a bit wrong': 13}, 'GT opposit': {'No, totally wrong': 15, 'yes, a bit better': 4, 'No, a bit wrong': 22, 'same': 5, 'yes, much better': 4}, 'Ours w/ user text': {'yes, a bit better': 18, 'No, a bit wrong': 8, 'yes, much better': 5, 'same': 15, 'No, totally wrong': 4}}

 ['go further away from combination lock', 'go much closer to Shih-Tzu', 'go to the bottom', 'go to oxygen mask', 'go much faster when passing snowmobile', 'go slower while passing plate', 'stay bottom', 'a lot faster', 'go slower when reaching Lakeland terrier'

In [44]:
print(num_users)
for dis in [dis_names[1],dis_names[-1]]:

    better = (total[dis]['yes, a bit better'] + total[dis]['yes, much better'])*100.0/(5*num_users)
    same = (total[dis]['same'])*100.0/(5*num_users)
    worse = (total[dis]['No, a bit wrong'] + total[dis]['No, totally wrong'])*100.0/(5*num_users)

    print(dis, "\t {:.2f} \t {:.2f} \t {:.2f}".format(better,same, worse) )

10
Ours 	 66.00 	 26.00 	 8.00
Ours w/ user text 	 46.00 	 30.00 	 24.00


In [66]:
unique_words = {}
for d in data:
    words = d["text"].split(" ")
    for w in words:
        unique_words[w]=1

print(len(unique_words.keys()))
user_unique_words = {}
for t in text_interactions:
    words = t.split(" ")
    for w in words:
        user_unique_words[w] = 1
print(len(user_unique_words.keys()))
print(user_unique_words.keys())
out_of_dis = {}
for w in user_unique_words.keys():
    if not w in unique_words.keys():
        out_of_dis[w] = 1
print(len(out_of_dis.keys()))
print(out_of_dis.keys())

print(len(out_of_dis.keys())/float(len(user_unique_words.keys())))

samples_out_of_dis = []
for t in text_interactions:
    words = t.split(" ")
    for w in words:
        if w in out_of_dis.keys():
            samples_out_of_dis.append(t)
            break

print(len(samples_out_of_dis))
print(len(samples_out_of_dis)/float(len(text_interactions)))
print(text_interactions)


1929
99
dict_keys(['go', 'further', 'away', 'from', 'combination', 'lock', 'much', 'closer', 'to', 'Shih-Tzu', 'the', 'bottom', 'oxygen', 'mask', 'faster', 'when', 'passing', 'snowmobile', 'slower', 'while', 'plate', 'stay', 'a', 'lot', 'reaching', 'Lakeland', 'terrier', 'top', 'snake', 'fence', 'water', 'ox', 'pier', 'seashore', 'ski', 'Hit', 'tabby', 'Surround', 'knee', 'pad', 'hit', 'and', 'avoid', 'space', 'bar', 'Go', 'coon', 'bear', 'near', 'kite', 'really', 'lacewing', 'fly', 'bit', 'around', 'palace', '', 'slow', 'down', 'abacus', 'move', 'close', 'hot', 'pot', 'front', 'far', 'sea', 'lower', 'upper', 'tray', 'knot', 'limo', 'do', 'not', 'pass', 'chiffonier', 'blue', 'jack', 'salt', 'shaker', 'envelope', 'Pass', 'next', 'power', 'grill', 'nearer', 'harvestman', 'Get', 'right', 'in', 'between', 'roundabout', 'Walk', 'head', 'left', 'use', 'direct', 'way', 'trike'])
23
dict_keys(['reaching', 'Hit', 'Surround', 'hit', 'avoid', 'Go', 'really', 'around', 'slow', 'move', 'far', 'lowe